In [1]:
import numpy as np

In [2]:
nodes=150
graphRandom = np.random.rand(nodes,nodes)

In [3]:
graphRandom[graphRandom>0.25]=1
graphRandom[graphRandom<=0.5]=0

graph=graphRandom*np.transpose(graphRandom)

np.fill_diagonal(graph,0)

In [4]:
# print(np.sum(graph,axis=1))
# graph

In [5]:
print("Number of edges ",sum(sum(graph>0))/2)
graph2 =np.array(graph)
for i in range(nodes):
    graph2[i] = graph2[i]/(np.sum(graph2[i])) 
# graph2

Number of edges  6304.0


In [6]:
def randomWalkFunction(graph,length,startNode):
    node=startNode
    itr=0
    path=[startNode]
    while (itr<length):
        validNextMoves=np.where(graph[node]>0)
        if len(validNextMoves[0])==0:
            break
        nextMove=int(np.floor(np.random.rand()*len(validNextMoves[0])))
        path.append(validNextMoves[0][nextMove])
        node=validNextMoves[0][nextMove]
        itr+=1
    return path

In [7]:
length=5000000
randomStart = int(np.floor(np.random.rand()*nodes))
randomWalk = randomWalkFunction(graph,length,randomStart)

In [14]:
windowSize=6
U=3
V=5
print(randomWalk[:10])

[40, 123, 20, 25, 148, 116, 3, 116, 146, 116]


In [15]:
#Q1 : Each ooccurance of random walk
pointersOfNodes={}

for i in range(nodes):
    pointersOfNodes[i]=[]
u=U
def occuranceOf(randomWalk,windowSize):
    context={}
    focus={}
    for i in range(nodes):
        context[i]=[]
        focus[i]=[]
    for i in range(windowSize,len(randomWalk)-windowSize):
        pointersOfNodes[randomWalk[i]].append(i)
        if randomWalk[i]==u:
            context[randomWalk[i]].append(np.array(randomWalk[i-windowSize:i+windowSize+1]))
            pointersOfNodes[randomWalk[i]].append(i)
            focus[randomWalk[i]].append(np.array(randomWalk[i-windowSize:i+windowSize+1]))
    return context,focus

context,focus=occuranceOf(randomWalk,windowSize)

nodesOccurance={}
for i in range(nodes):
    if i != u:
        continue
    count=0
    temp=context[i]
    for cont in temp:
        count+=len(cont)-1
    temp=focus[i]
    for cont in temp:
        count+=len(cont)-1
    
    nodesOccurance[i]=float(count)/len(temp)
    

In [28]:
#Q2: Probablity of walk starting at node u will vist node v
u=U
v=V
t=windowSize
def probablityOfPath(u,v,randomWalk,t,pointersOfNodes):
    count=0
    for ptr in pointersOfNodes[u]:
        if randomWalk[ptr+t]==v:
            count+=1
    return float(count)/len(pointersOfNodes[u]),float(count),len(pointersOfNodes[u])

print(probablityOfPath(u,v,randomWalk,t,pointersOfNodes))

def theoretical(u,v,t,graph2):
    oneHot=[0]*nodes
    oneHot[u]=1
    oneHot = np.transpose(oneHot)
#     np.linalg.matrix_power(graph2,t)
    return np.dot(oneHot,np.linalg.matrix_power(graph2,t))

print(theoretical(u,v,t,graph2)[v])

(0.005983840592916591, 197.0, 32922)
0.00658305176625


In [30]:
#Q4B Intuition: Probablity of walk starting at node u will vist node v in left window
u=U
v=V
U=2
V=8
t=5
# t=windowSize
def probablityOfPath(u,v,randomWalk,t,pointersOfNodes):
    count=0
    for ptr in pointersOfNodes[u]:
        if randomWalk[ptr-t]==v:
            count+=1
    return float(count)/len(pointersOfNodes[u]),float(count),len(pointersOfNodes[u])

print(probablityOfPath(u,v,randomWalk,t,pointersOfNodes))

def theoretical(u,v,t,graph2):
    oneHot=[0]*nodes
    oneHot[u]=1
    oneHot = np.transpose(oneHot)
#     np.linalg.matrix_power(graph2,t)
    return np.dot(oneHot,np.linalg.matrix_power(graph2,t))

print(theoretical(v,u,t,graph2)[v])

(0.005771216815503311, 190.0, 32922)
0.00610631296662


In [11]:
randomWalk[0:5]

[40, 123, 20, 25, 148]

In [17]:
#Q3 Expected number of times node v is visited with t steps of Node U
u=U
v=V
t=windowSize
def ExpectedSimulation(u,v,randomWalk,windowSize,pointersOfNodes):
    count=0
    for ptr in pointersOfNodes[u]:
        if v in randomWalk[ptr:ptr+windowSize+1]:
            count+=1
    return float(count)/len(pointersOfNodes[u]),float(count),len(pointersOfNodes[u])

print(ExpectedSimulation(u,v,randomWalk,t,pointersOfNodes))

def ExpectedTheoretical(u,v,T,graph2):
    count=0
    for t in range(1,T):
        count+=theoretical(u,v,t,graph2)[v]
    return count
print(ExpectedTheoretical(u,v,t,graph2))

(0.042021422686075255, 2448.0, 58256)
0.0397713247685


In [18]:
#Question 4: Skipgram softmax
t=windowSize
u=U
v=V

def beforeTStepOccurance(u,v,randomWalk,t,pointersOfNodes):
    count=0
    for ptr in pointersOfNodes[u]:
        if v == randomWalk[ptr-t]:
            count+=1
    return float(count)/len(pointersOfNodes[u]),float(count),len(pointersOfNodes[u])

def withinWindow(u,v,randomWalk,windowSize,pointersOfNodes):
    count=0
    for ptr in pointersOfNodes[u]:
        if v in randomWalk[ptr-windowSize:ptr+windowSize+1]:
            count+=np.sum(np.array(randomWalk[ptr-windowSize:ptr+windowSize+1])==v)
          
    return float(count)/(2* windowSize*len(pointersOfNodes[u])),float(count),len(pointersOfNodes[u])

prob,countUandV,uCount=withinWindow(u,v,randomWalk,t,pointersOfNodes)


withinTSteps = ExpectedTheoretical(u,v,t,graph2)
beforeTSteps =0
for t in range(1,windowSize):
    beforeTSteps += beforeTStepOccurance(u,v,randomWalk,t,pointersOfNodes)[0]
print(prob)
print(withinTSteps,beforeTSteps)
print((withinTSteps+beforeTSteps)/(2*windowSize))
print(withinTSteps/float(windowSize))

0.007564313833196008
0.0397713247685 0.04064817357868718
0.00670162486227
0.00662855412808


In [ ]:
USet=[3,6,7,8,67,57]
VSet=[1,5,4,5,6,7,8]
WindowSizeArray=[2,3,4,5,6]

# USet=[3]
# VSet=[4]
# WindowSizeArray=[4]

for u1 in USet:
    for v1 in VSet:
        for w in WindowSizeArray:
            print(u1,v1,w)
            if u1 ==v1 or u1 >= nodes or v1 >=nodes:
                continue
            u=u1
            v=v1
            t=w
            print("Q2")
            print("Simulation ",probablityOfPath(u,v,randomWalk,t,pointersOfNodes)[0])
            print(theoretical(u,v,t,graph2)[v])
            
            print("===========================")
            print("Q3")
            print("Simulation ",ExpectedSimulation(u,v,randomWalk,t,pointersOfNodes)[0])
            print(ExpectedTheoretical(u,v,t,graph2))
            
            print("===========================")
            print("Q4")
            
            prob,countUandV,uCount=withinWindow(u,v,randomWalk,t,pointersOfNodes)
            
            withinTSteps = ExpectedTheoretical(u,v,t,graph2)
            beforeTSteps =0
            for t in range(1,windowSize):
                beforeTSteps += beforeTStepOccurance(u,v,randomWalk,t,pointersOfNodes)[0]
            
            print ("Simulation   ", prob)
            print ("Solution1    ", (beforeTSteps+withinTSteps)/(2*t))
            print ("P(u,v,t)/t   " ,withinTSteps/float(t) )
            
            print("\n------------------------------------\n\n\n")
